# TESTING ENVS

In [9]:
from myosuite.utils import gym
import skvideo.io
import numpy as np
import os
from stable_baselines3 import PPO
from tqdm.notebook import tqdm

In [ ]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 400):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")
 

In [10]:
env = gym.make('myoHandWheelHoldFixed-v0')
env.reset()

(array([ 0.    ,  0.0339,  0.0229, -0.011 , -0.465 ,  0.1272, -0.0095,
         0.    , -0.152 ,  0.5833,  0.    , -0.552 , -0.3131,  0.    ,
        -0.2639,  0.4085, -0.37  ,  2.094 ,  1.0778,  0.11  ,  0.4363,
        -0.7854,  0.78  , -0.6246,  0.1937,  0.0087,  1.1704,  0.2356,
         0.8955,  1.3432,  1.461 ,  0.0105,  0.7384,  1.1625,  1.3354,
        -0.1545,  0.1414,  1.571 ,  1.2411, -0.2618,  0.4477,  0.597 ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
         0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,
      

In [ ]:
model = PPO("MlpPolicy", env, verbose=0)

In [ ]:
model.learn(total_timesteps=1e4)
# to train to convergence use more iterations e.g.
# model.learn(total_timesteps=1e7)

In [ ]:
model.save("WheelDist_policy")

In [ ]:
# env.sim.renderer.set_viewer_settings(
#            render_actuator=True,
#            render_tendon=True
#        )

pi = PPO.load("WheelDist_policy")

env = gym.make('myoHandWheelHoldFixed-v0')
env.reset()

In [ ]:
# render
env.reset()
frames = []
for _ in range(300):
    frames.append(env.sim.renderer.render_offscreen(width=400, height=400, camera_id=3))
    o = env.get_obs()
    a = pi.predict(o)[0]
    next_o, r, done, *_, ifo = env.step(
        a
    )  # take an action based on the current observation

import skvideo.io
import os
os.makedirs("videos", exist_ok=True)
# make a local copy
skvideo.io.vwrite(
    "videos/RockPose.mp4",
    np.asarray(frames),
    outputdict={"-pix_fmt": "yuv420p", "-r": "10"},
)
show_video('videos/RockPose.mp4')

In [ ]:
# evaluate policy
all_rewards = []
for _ in tqdm(range(5)): # 5 random targets
  ep_rewards = []
  done = False
  obs = env.reset()
  done = False
  for _ in range(100):
      obs = env.obsdict2obsvec(env.obs_dict, env.obs_keys)[1]
      # get the next action from the policy
      action, _ = model.predict(obs, deterministic=True)
      # take an action based on the current observation
      obs, reward, done, info, _ = env.step(action)
      ep_rewards.append(reward)
  all_rewards.append(np.sum(ep_rewards))
print(f"Average reward: {np.mean(all_rewards)} over 5 episodes")
all_rewards

# Python testing

In [7]:
import mujoco

In [4]:
model = mujoco.MjModel.from_xml_path("../assets/wheelchair/myowc+arm.xml")
data = mujoco.MjData(model)

In [5]:
data.qpos[model.joint_name2id("right_rear")]

AttributeError: 'mujoco._structs.MjModel' object has no attribute 'joint_name2id'

In [ ]:
model.ngeom

In [ ]:
model.geom_rgba

In [ ]:
[model.geom(i).name for i in range(model.ngeom)]

## `mjData`
`mjData` contains the *state* and quantities that depend on it. The state is made up of time, [generalized](https://en.wikipedia.org/wiki/Generalized_coordinates) positions and generalized velocities. These are respectively `data.time`, `data.qpos` and `data.qvel`. In order to make a new `mjData`, all we need is our `mjModel`

In [ ]:
[model.site(i).name for i in range(model.nsite)]

In [ ]:
print(data.site_xpos) #XYZ positions of sites listed above

In [8]:
# JOINTS
print([model.jnt(i).name for i in range(model.njnt)])
print(model.njnt)

#joint position
print(data.qpos)
print(len(data.qpos))

#joint velocity
print(data.qvel)
print(len(data.qvel))

['wheelchair', 'left_rear', 'right_rear', 'left_fork', 'left_caster', 'right_fork', 'right_caster', 'sternoclavicular_r2', 'sternoclavicular_r3', 'unrotscap_r3', 'unrotscap_r2', 'acromioclavicular_r2', 'acromioclavicular_r3', 'acromioclavicular_r1', 'unrothum_r1', 'unrothum_r3', 'unrothum_r2', 'elv_angle', 'shoulder_elv', 'shoulder1_r2', 'shoulder_rot', 'elbow_flexion', 'pro_sup', 'deviation', 'flexion', 'cmc_abduction', 'cmc_flexion', 'mp_flexion', 'ip_flexion', 'mcp2_flexion', 'mcp2_abduction', 'pm2_flexion', 'md2_flexion', 'mcp3_flexion', 'mcp3_abduction', 'pm3_flexion', 'md3_flexion', 'mcp4_flexion', 'mcp4_abduction', 'pm4_flexion', 'md4_flexion', 'mcp5_flexion', 'mcp5_abduction', 'pm5_flexion', 'md5_flexion']
45
[0.     0.     0.0031 0.9999 0.0127 0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.  

In [ ]:
# number of actuators, we don't have any because no muscles
model.na 

In [ ]:
data.act 
#action vector (muscle activations or actuator commands).